In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import findspark
findspark.init()
import pyspark as spark
from pyspark.sql.functions import udf,col, rand
from pyspark.sql.types import StringType

In [ ]:
spark.SparkConf()

In [ ]:
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

In [ ]:
df = sql.read.option("delimiter",'\t').option("header",'true').csv('GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_reads.gct')
df = df.withColumn('ensg',udf(lambda x: x[:15], StringType())(col('Name')))
df.registerTempTable('gtex')

In [ ]:
genes = sql.read.option("header",'true').csv('/Users/filippo/Developer/tesi/genes.txt')
genes.registerTempTable("genes")
genes.select("type_of_gene").distinct().show()

In [ ]:
pc = sql.sql("SELECT * FROM genes WHERE type_of_gene='protein-coding'")
pc_list = list(pc.select('_c0').orderBy(rand()).toPandas().values.T[0])

In [ ]:
df = df.filter(col("ensg").isin(pc_list))

In [ ]:
df.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("mainTable.csv")

In [ ]:
df.toPandas().to_csv("mainTable.csv", index=True, header=True)

## GTEx Biospecimen
[GTEX_biospecimen](https://gtexportal.org/home/samplesPage)

In [ ]:
#df_file = sql.createDataFrame(pd.read_csv("https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_v7_Annotations_SampleAttributesDS.txt", sep='\t', index_col=[0]).loc[:,['SMTS', 'SMTSD']])
df_file=sql.read.option('delimiter',',').option("header",'true').csv("files.dat")
df_file.registerTempTable("files")
df_file.describe()

In [ ]:
#df_sites=pd.DataFrame()
#df_sites.insert(0,'sample',df_file.index.values)
#df_sites.insert(1,'primary_site',df_file.loc[:,'SMTS'].values)
#df_sites.insert(2,'secondary_site', df_file.loc[:,'SMTSD'].values)
#df_sites.set_index('sample', inplace=True)
#df_sites.head()

In [ ]:
df_file.select('primary_site').distinct().show()

In [ ]:
available_tissues=df_file.groupby('primary_site').count().sort("count", ascending=False).toPandas()
available_tissues

In [ ]:
#selectedTissues = ['Blood', 'Brain', 'Lung', 'Thyroid', 'Breast', 'Skin', 'Pancreas', 'Testis', 'Adipose Tissue', 'Colon', 'Nerve', 'Heart', 'Kidney', 'Uterus', 'Adrenal Gland']
selectedTissues = available_tissues['primary_site'][:10].values
selectedTissues

In [ ]:
limit=int(1000/len(selectedTissues))
needed_columns = []
for tissue in selectedTissues:
    needed_columns = np.concatenate((needed_columns,
                                 list(set(sql.sql("SELECT * FROM files WHERE primary_site='%s'"%(tissue))
                                .orderBy(rand())
                                .limit(limit)
                                .select("_c0")
                                .toPandas().values.T[0])-{'Name'}-{'Description'})))
np.random.shuffle(needed_columns)
needed_columns = np.concatenate([['ensg'],needed_columns])

In [ ]:
df.select(list(needed_columns)).toPandas().to_csv("mainTable_all.csv", header=True, index=False)

In [ ]:
df_file.filter(col('_c0').isin(list(needed_columns))).toPandas().to_csv("files_tissues.dat", header=True, index=False)

# U

In [ ]:
tissue = 'Breast'

In [ ]:
sample_list = df_sites[df_sites['primary_site']==tissue].index.values
subdf = df.loc[:,sample_list]

In [ ]:
odf = pd.read_csv("meanVariances.csv", header=0, index_col=0)
odf.insert(3,"A",pd.read_csv("A.dat", header=None).values)
odf.head()

In [ ]:
O = odf[odf.index.isin(pc.index)]['occurrence']
O_nc = odf[~odf.index.isin(pc.index)]['occurrence']

In [ ]:
bins = 20
rang = (0-0.5/20,1+0.5/20)
fig = plt.figure()
plt.hist(O[O<=1], histtype='step', lw=4, density=True, bins=bins, range=rang, label='coding')
plt.hist(O_nc[O_nc<=1], histtype='step', lw=4, density=True, bins=bins, range=rang, label='non-coding')
#plt.title(tissue, fontsize=18)
plt.xlabel('$O_i$', fontsize=16)
plt.ylabel('pdf', fontsize=16)
plt.legend(loc='upper left', fontsize=16)
plt.show()
fig.savefig("U_gtex_cnc.pdf")

In [ ]:
A = odf['A'].values
A_c = odf[odf.index.isin(pc.index)]['A'].values
A_nc = odf[~odf.index.isin(pc.index)]['A'].values
fig=plt.figure()
plt.plot(np.arange(0,len(A)), 1./np.arange(0,len(A)), ls='--', c='g', label='$r^-1$')
#plt.plot(np.arange(0,len(A)), np.sort(A)[::-1]/np.sum(A))
#plt.plot(np.arange(0,len(A_c)), np.sort(A_c)[::-1]/np.sum(A), label='coding genes')
plt.plot(np.arange(0,len(A_nc)), np.sort(A_nc)[::-1]/np.sum(A), label='non coding genes')
plt.xscale('log')
plt.yscale('log')
plt.ylabel('$f_i$', fontsize=18)
plt.xlabel("Rank_i", fontsize=18)
plt.legend(fontsize=18)
fig.savefig("globalZipf_nc.pdf")